In [1]:
import requests
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as BS
import pandas as pd

## Obtaining URLs using API and Webscrapping

In [2]:
app_id= "e7cbde9e-8241-4e59-a78a-9c866ff45eb9"
my_key= "meZeRjFlLqRyFEAGOEIKcW0mkiWRIXdJ"

# function to aggregatede the URLs from the API
def call_nyt_by_year_and_page(year, page):
    root_url   = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=tesla&sort=newest'
    begin_date = '&begin_date={}0101'.format(str(year))
    end_date   = '&end_date={}1231'.format(str(year))
    pagination = '&page={}'.format(str(page))
    doc_params = '&fl=web_url&fl=snippet&fl=pub_date&fl=_id&fl=lead_paragraph'
    
    url        = root_url + begin_date + end_date + pagination + doc_params + '&api-key=' + my_key
    print(url)
    
    response   = requests.get(url)
    
    return response.json()

In [3]:
# returns the articles

def call_nyt_by_year(year):
    annual_articles = []
    
    # find how many articles (hits) contain 'tesla' in a calendar year
    total_results = call_nyt_by_year_and_page(year,0)
    hits  = total_results['response']['meta']['hits']
    
    # request all available pages
    # sleep 7 seconds to clear NYT API rate limit
    for i in range(int(hits/10)):
        query = call_nyt_by_year_and_page(year,i)
        annual_articles = annual_articles + query['response']['docs']
        time.sleep(7)
        
    return annual_articles

In [4]:
## aggregates all the articles
def call_nyt_by_all_years(years):
    all_articles = []
    
    for year in years:
        annual_articles = call_nyt_by_year(year)
        all_articles = all_articles + annual_articles
        time.sleep(7)
        
    return all_articles

In [ ]:
# provides a range for the years that I want to include in my project

#takes a while to run

all_years    = list(range(2013,2021))
all_articles = call_nyt_by_all_years(all_years)

### ran into some troubles while webscrapping so I had to break the scrapping into two parts, here I loaded the pre saved data from the first time webscrape

In [5]:
# opening data


with open("nyt_api_2010_2019.json", "r") as read_file:
    data = json.load(read_file)

In [10]:
data

[{'web_url': 'https://www.nytimes.com/2011/01/02/automobiles/02WRAP2010.html',
  'snippet': 'For the automotive industry, 2010 played out like a reality show, with heroes and villains, surprising alliances, narrow escapes and thrilling reversals of fortune.',
  'lead_paragraph': 'THE last year in automobiles could have been mistaken for an episode of “Survivor,” only set in the urban jungle of Detroit rather than some tropical backwater. ',
  'pub_date': '2010-12-31T13:58:12+0000',
  '_id': '5c9eafb249f0eacbf11661b4',
  'article': 'AdvertisementSupported byByLawrence UlrichTHE last year in automobiles could have been mistaken for an episode of “Survivor,” only set in the urban jungle of Detroit rather than some tropical backwater.As industry players retrenched and gritted their teeth for challenges to come, the 2010 season featured heroes and villains, new alliances, narrow escapes and thrilling reversals of fortune.For some played-out brands, there was no room on the shrinking island.

In [6]:
## Beautiful soup that scrapes the body paragraphs

def get_nyt_text(url):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    page = requests.get(url, headers=headers)
    page.status_code
    
    soup = BS(page.content, 'html.parser')
    content = soup.find_all('p')
    
    nyt = ''
    for index in range(len(content)):
        nyt += content[index].get_text(strip= True)
        
    return nyt

In [ ]:
## Add the body paragraph to the dictionary of Data DF

for i in range(len(data)):
    print(i)
#     print(data[i]['web_url'])
    data[i]['article'] = get_nyt_text(data[i]['web_url'])

In [ ]:
import re

## using regex to strip the Advertisements that were taken from beautiful soup
def dict_cleaning(data):
    for dic in data:
        for num in range(len(data)):
            s=dic['article'].encode("ascii", "ignore").decode("utf-8")
            s=re.sub("\n","", s)
            s=re.sub("\s{4,}"," ",s)
            s=re.sub("(^|Advert)[A-Za-z]*\s(\w+)\s(\w+)","",s)
            # s=re.sub("AdvertisementSupported byBy","",s)
            s=re.sub(".Advertisement","",s)
            dic['article']=s
  

In [ ]:
dict_cleaning(data)

In [ ]:
## open the second part of articles from 2019- on

with open("nyt_tesla_scrapped_2019_on", "r") as read_file:
    data_2019 = json.load(read_file)

In [ ]:
dict_cleaning(data_2019)

In [9]:
#clearly not all articles mentions the word tesla in them so lets gather a full lsit of articles from both scrapped sets

word="Tesla"
articles=[]
for i in range(len(data)):
    if word in data[i]['article']:
        articles.append(data[i])
#now for the 2019 dataset        
for i in range(len(data_2019)):
    if word in data_2019[i]['article']:
        articles.append(data[i])
        
len(articles)

KeyError: 'article'

In [ ]:
#save the article file so we do not have run into any issues
with open("selected_articles", "w") as write_file:
    json.dump(articles, write_file)